In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print("CUDA_VISIBLE_DEVICES:", os.environ["CUDA_VISIBLE_DEVICES"])

In [1]:
from plot_utils import *

def plot_figures(energy, samples, name=None):
  
  if 'many_well' in name:
    
    vizualizations = viz_many_well(energy, samples)
    fig_samples_x13, ax_samples_x13, fig_kde_x13, ax_kde_x13, fig_contour_x13, ax_contour_x13, fig_samples_x23, ax_samples_x23, fig_kde_x23, ax_kde_x23, fig_contour_x23, ax_contour_x23 = vizualizations
    
  elif name=='25gmm':
    batch_size = samples.shape[0]
    gt_samples = energy.sample(batch_size)

    fig_contour, ax_contour = get_figure(bounds=(-20., 20.))
    fig_kde, ax_kde = get_figure(bounds=(-20., 20.))
    fig_kde_overlay, ax_kde_overlay = get_figure(bounds=(-20., 20.))

    plot_contours(energy.log_reward, ax=ax_contour, bounds=(-20., 20.), n_contour_levels=50, device=samples.device)
    plot_kde(gt_samples, ax=ax_kde_overlay, bounds=(-20., 20.))
    plot_kde(samples, ax=ax_kde, bounds=(-20., 20.))
    plot_samples(samples, ax=ax_contour, bounds=(-20., 20.))
    plot_samples(samples, ax=ax_kde_overlay, bounds=(-20., 20.))
    
  elif name == '40gmm':
    batch_size = samples.shape[0]
    gt_samples = energy.sample(batch_size)

    fig_contour, ax_contour = get_figure(bounds=(-50., 50.))
    fig_kde, ax_kde = get_figure(bounds=(-50., 50.))
    fig_kde_overlay, ax_kde_overlay = get_figure(bounds=(-50., 50.))

    plot_contours(energy.log_reward, ax=ax_contour, bounds=(-50., 50.), n_contour_levels=50, device=samples.device)
    plot_kde(gt_samples, ax=ax_kde_overlay, bounds=(-50., 50.))
    plot_kde(samples, ax=ax_kde, bounds=(-50., 50.))
    plot_samples(samples, ax=ax_contour, bounds=(-50., 50.))
    plot_samples(samples, ax=ax_kde_overlay, bounds=(-50., 50.))

In [2]:
def draw_energy_histogram(
    ax, log_reward, bins=40, range=(90, 160)
):
    log_reward = torch.clamp(log_reward, min=range[0], max=range[1])

    hist, bins = np.histogram(
        log_reward.detach().cpu().numpy(), bins=bins, range=range, density=True
    )

    ax.set_xlabel("log reward")
    ax.set_ylabel("count")
    ax.grid(True)

    return ax.plot(bins[1:], hist, linewidth=3)

In [3]:
def draw_interatomic_dist_histogram(
    ax, interatomic_dist, name=None, bins=40, range=(0, 3)
):
    interatomic_dist = torch.clamp(interatomic_dist, min=range[0], max=range[1])

    hist, bins = np.histogram(
        interatomic_dist.detach().cpu().numpy(), bins=bins, range=range, density=True
    )

    ax.set_xlabel("Interatomic Distances")
    ax.set_ylabel("Normalized density")
    ax.grid(True)

    return ax.plot(bins[1:], hist, label=name, linewidth=3)

In [4]:
import argparse

parser = argparse.ArgumentParser(description='Model Load')

parser.add_argument('--teacher', type=str, default=None, choices=('ais', 'mala'))
parser.add_argument('--round', type=int, default=1)

parser.add_argument('--lr_policy', type=float, default=1e-3)
parser.add_argument('--lr_flow', type=float, default=1e-3)
parser.add_argument('--lr_back', type=float, default=1e-3)
parser.add_argument('--hidden_dim', type=int, default=256)
parser.add_argument('--s_emb_dim', type=int, default=256)
parser.add_argument('--t_emb_dim', type=int, default=256)
parser.add_argument('--harmonics_dim', type=int, default=64)
parser.add_argument('--batch_size', type=int, default=300)
parser.add_argument('--epochs', type=int, default=25000)
parser.add_argument('--buffer_size', type=int, default=600000)
parser.add_argument('--T', type=int, default=100)
parser.add_argument('--subtb_lambda', type=int, default=2)
parser.add_argument('--t_scale', type=float, default=1.)
parser.add_argument('--log_var_range', type=float, default=4.)
parser.add_argument('--energy', type=str, default='9gmm',
                    choices=('9gmm', '25gmm', '40gmm', 'hard_funnel', 'easy_funnel', 'many_well_32', 'many_well_64', 'many_well_128',
                             'many_well_512'))
parser.add_argument('--mode_fwd', type=str, default="tb", choices=('tb', 'tb-avg', 'db', 'subtb', "pis"))
parser.add_argument('--mode_bwd', type=str, default="tb", choices=('tb', 'tb-avg', 'mle'))
parser.add_argument('--both_ways', action='store_true', default=False)

# For local search
################################################################
parser.add_argument('--local_search', action='store_true', default=False)

# How many iterations to run local search
parser.add_argument('--max_iter_ls', type=int, default=200)

# How many iterations to burn in before making local search
parser.add_argument('--burn_in', type=int, default=100)

# How frequently to make local search
parser.add_argument('--ls_cycle', type=int, default=100)

# langevin step size
parser.add_argument('--ld_step', type=float, default=0.001)

parser.add_argument('--ld_schedule', action='store_true', default=False)

# target acceptance rate
parser.add_argument('--target_acceptance_rate', type=float, default=0.574)


# For replay buffer
################################################################
# high beta give steep priorization in reward prioritized replay sampling
parser.add_argument('--beta', type=float, default=1.)

# low rank_weighted give steep priorization in rank-based replay sampling
parser.add_argument('--rank_weight', type=float, default=1e-2)

# three kinds of replay training: random, reward prioritized, rank-based
parser.add_argument('--prioritized', type=str, default="rank", choices=('none', 'reward', 'rank'))
################################################################

parser.add_argument('--bwd', action='store_true', default=False)
parser.add_argument('--exploratory', action='store_true', default=False)

parser.add_argument('--sampling', type=str, default="buffer", choices=('sleep_phase', 'energy', 'buffer'))
parser.add_argument('--langevin', action='store_true', default=False)
parser.add_argument('--langevin_scaling_per_dimension', action='store_true', default=False)
parser.add_argument('--conditional_flow_model', action='store_true', default=False)
parser.add_argument('--learn_pb', action='store_true', default=False)
parser.add_argument('--pb_scale_range', type=float, default=0.1)
parser.add_argument('--learned_variance', action='store_true', default=False)
parser.add_argument('--partial_energy', action='store_true', default=False)
parser.add_argument('--exploration_factor', type=float, default=0.1)
parser.add_argument('--exploration_wd', action='store_true', default=False)
parser.add_argument('--clipping', action='store_true', default=False)
parser.add_argument('--lgv_clip', type=float, default=1e2)
parser.add_argument('--gfn_clip', type=float, default=1e4)
parser.add_argument('--zero_init', action='store_true', default=False)
parser.add_argument('--pis_architectures', action='store_true', default=False)
parser.add_argument('--lgv_layers', type=int, default=3)
parser.add_argument('--joint_layers', type=int, default=2)
parser.add_argument('--seed', type=int, default=12345)
parser.add_argument('--weight_decay', type=float, default=1e-7)
parser.add_argument('--use_weight_decay', action='store_true', default=False)
parser.add_argument('--eval', action='store_true', default=False)
args = parser.parse_args(args=[])

In [ ]:
import pandas as pd
# many_well_64
data_mw_64 = {
    'Method': [
        'PIS+LP', 'TB+LP', 'SubTB+LP', 'TB+LS+LP', 'TB+Expl+LP', 'TB+Expl+LS+LP', 
        'PIS', 'TB', 'TB+LS', 'TB+Expl+LS', 
        'Ours'
    ],
    'ELBO': [
        310.36, 306.28, 316.05, 312.66, 306.45, 299.85,
        321.87, 317.35, 318.35, 265.99,
        321.48
    ],
    'ELBO_std': [
        0.71, 0.27, 6.92, 2.66, 0.31, 1.23,
        0.05, 6.01, 30.00, 95.39,
        0.19
    ],
    'EUBO': [
        347.02, 351.58, 368.44, 339.34, 351.26, 344.85,
        2026.78, 853.94, 360.00, 361.00,
        335.72
    ],
    'EUBO_std': [
        2.10, 0.66, 7.35, 1.02, 0.64, 0.43,
        499.73, 43.35, 17.00, 16.58,
        0.11
    ],
    'EnergyCalls': [
        366458300, 570458300, 1135488300, 586988300, 570458300, 586988300,
        112030000, 127232000, 300179500, 300179500,
        109853600
    ]
}

df_mw_64 = pd.DataFrame(data_mw_64)
df_mw_64


In [ ]:
markers = ['o', 's', '^', 'v', 'D', 'X', 'h', 'H', 'p', 'P', '*']  # 필요 시 추가
colors = ['olive', 'red', 'green', 'cyan', 'magenta', 'orange', 'purple', 'brown', 'pink', 'gray', 'blue']

methods = df_mw_64['Method'].unique()

plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(14, 6))


plt.subplot(1, 2, 1)
plt.title("Many-well 64")

for i, method in enumerate(methods):
    sub_df = df_mw_64[df_mw_64['Method'] == method]
    plt.errorbar(
        sub_df['EnergyCalls'],
        sub_df['ELBO'],
        # yerr=sub_df['ELBO_std'],
        fmt=markers[i],        
        label=method,
        capsize=5,
        markersize=8,
        linestyle='None',
        color=colors[i]            
    )

plt.xlabel("Energy Calls")
plt.ylabel("ELBO")
plt.legend(loc='best')
plt.grid(alpha=0.3)
# 필요한 경우 x축 로그 스케일로 설정: plt.xscale('log')

plt.subplot(1, 2, 2)
plt.title("Many-well 64")

for i, method in enumerate(methods):
    sub_df = df_mw_64[df_mw_64['Method'] == method]
    plt.errorbar(
        sub_df['EnergyCalls'],
        sub_df['EUBO'],
        # yerr=sub_df['EUBO_std'],
        fmt=markers[i],
        label=method,
        capsize=5,
        markersize=8,
        linestyle='None',
        color=colors[i]
    )

plt.xlabel("Energy Calls")
plt.ylabel("EUBO")
plt.legend(loc='best')
plt.grid(alpha=0.3)
# plt.xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
# many_well_128
data_mw_128 = {
    'Method': [
        'PIS+LP', 'TB+LP', 'SubTB+LP', 'TB+LS+LP', 'TB+Expl+LP', 'TB+Expl+LS+LP', 
        'PIS', 'TB', 'TB+LS', 'TB+Expl+LS', 
        'Ours'
    ],
    'ELBO': [
        599.87, 612.45, 609.01, 592.52, 612.37, 591.47,
        643.29, 495.20, 496.20, 589.49, 
        621.29
    ],
    'ELBO_std': [
        0.94, 0.65, 0.31, 2.25, 0.65, 0.36,
        0.10, 200.53, 8.00, 7.25,
        4.34
    ],
    'EUBO': [
        696.63, 706.73, 710.69, 693.65, 707.91, 694.93,
        1218.44, 1040.26, 699.00, 698.24, 
        681.16
    ],
    'EUBO_std': [
        0.53, 2.59, 5.00, 1.40, 1.89, 0.54,
        124.44, 293.88, 3.00, 2.81,
        0.92
    ],
    'EnergyCalls': [
        366458300, 570458300, 1135488300, 586988300, 570458300, 586988300,
        112030000, 127232000, 300179500, 300179500,
        109853600
    ]
}

df_mw_128 = pd.DataFrame(data_mw_128)
df_mw_128

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

markers = ['o', 's', '^', 'v', 'D', 'X', 'h', 'H', 'p', 'P', '*']  # 필요 시 추가
colors = ['olive', 'red', 'green', 'cyan', 'magenta', 'orange', 'purple', 'brown', 'pink', 'gray', 'blue']

methods = df_mw_128['Method'].unique()

plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(14, 6))


plt.subplot(1, 2, 1)
plt.title("Many-well 128")

for i, method in enumerate(methods):
    sub_df = df_mw_128[df_mw_128['Method'] == method]
    plt.errorbar(
        sub_df['EnergyCalls'],
        sub_df['ELBO'],
        # yerr=sub_df['ELBO_std'],
        fmt=markers[i],        
        label=method,
        capsize=5,
        markersize=8,
        linestyle='None',
        color=colors[i]            
    )

plt.xlabel("Energy Calls")
plt.ylabel("ELBO")
plt.legend(loc='best')
plt.grid(alpha=0.3)
# 필요한 경우 x축 로그 스케일로 설정: plt.xscale('log')

plt.subplot(1, 2, 2)
plt.title("Many-well 128")

for i, method in enumerate(methods):
    sub_df = df_mw_128[df_mw_128['Method'] == method]
    plt.errorbar(
        sub_df['EnergyCalls'],
        sub_df['EUBO'],
        # yerr=sub_df['EUBO_std'],
        fmt=markers[i],
        label=method,
        capsize=5,
        markersize=8,
        linestyle='None',
        color=colors[i]
    )

plt.xlabel("Energy Calls")
plt.ylabel("EUBO")
plt.legend(loc='best')
plt.grid(alpha=0.3)
# plt.xscale('log')

plt.tight_layout()
plt.show()

In [ ]:
import glob
import torch

# 특정 디렉토리 내 모든 .pt 파일 경로 가져오기
pt_files = glob.glob("../sample_eff_check_points/many_well_128/sample_eff/current_ours/60_30/seed_12345/*.pt")

# 데이터를 저장할 리스트
data = []

# 모든 .pt 파일 로드
for pt_file in pt_files:
    try:
        metrics = torch.load(pt_file)  # .pt 파일 로드
        data.append(metrics)          # 데이터를 리스트에 추가
    except Exception as e:
        print(f"Error loading {pt_file}: {e}")

print(f"Loaded {len(data)} .pt files.")

In [ ]:
filtered_data = []

for item in data:
    filtered_item = {
        'energy_call_count': item['energy_call_count'],
        'eval/ELBO': item['metrics']['eval/ELBO'],
        'eval/EUBO': item['metrics']['eval/EUBO']
    }
    filtered_data.append(filtered_item)

# 결과 확인
print(filtered_data)

In [ ]:
# 각 요소에 'EUBO_minus_ELBO' 값을 추가
for item in filtered_data:
    eubo = item['eval/EUBO']
    elbo = item['eval/ELBO']
    item['EUBO_minus_ELBO'] = eubo - elbo

# 결과 확인
print(filtered_data)

In [ ]:
import matplotlib.pyplot as plt

# x와 y 값 추출
x = [item['energy_call_count'] for item in filtered_data]
y = [item['EUBO_minus_ELBO'].item() for item in filtered_data]  # tensor 값을 숫자로 변환

# Plot 그리기
plt.figure(figsize=(10, 6))
plt.scatter(x, y, color='blue', alpha=0.7, label='EUBO - ELBO')
plt.title("Energy Call Count vs EUBO - ELBO")
plt.xlabel("Energy Call Count")
plt.ylabel("EUBO - ELBO")
plt.xscale('log')
plt.grid(alpha=0.3)
plt.legend()
plt.show()